<a href="https://colab.research.google.com/github/Jihyun13579/ESAA/blob/main/1006(%EA%B8%88)_%EA%B3%BC%EC%A0%9C_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_4_p_487_~_496_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1006(금) 과제_파이썬 머신러닝 완벽 가이드 ch8. 4 p. 487 ~ 496 필사
### 2129014 박지현

## CH08.04. 텍스트 분류 실습 - 20 뉴스그룹 분류
- 사이킷런 : fetch_20newsgroups() API 이용해 뉴스 그룹의 분류 수행해볼 수 있는 예제 데이터 제공

- 텍스트 > 피처 벡터화로 변환 : 희소 행렬 형태
- 희소행렬 효과적으로 잘 처리할 수 있는 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 > 로지스틱 회귀 이용해 분류 수행

1. 텍스트 정규화
2. 피처 벡터화
 - 카운트 기반 vs TF-IDF 기반 벡터화
3, 적합한 머신러닝 알고리즘 적용해 분류 학습/예측/평가

### 텍스트 정규화
- fetch_20newsgroups() : 파이썬 딕셔너리와 유사한 Bunch 객체 반환

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 156)

- key 값 확인하기

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


'filenames' : 인터넷에서 내려받아 로컬 컴퓨터에서 저장하는 디렉터리와 파일명 지칭

- Target 클래스의 구성 확인하기

In [3]:
import pandas as pd
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


> target 클래스: 0~ 19 > 20개로 구성

- 개별 데이터가 텍스트로 어떻게 구성되어 있는지 데이터 한 개만 추출해 값 확인하기

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

> 뉴스 그룹 기사 내용뿐만 아니라 뉴스 그룹 제목, 작성자, 소속, 이메일 등 다양한 정보 가지고 있음 > 내용 제외 제목 등의 다른 정보 제거 필요 > 뉴스 그룹 분류의 Target 클래스 값과 유사한 데이터 가지고 있는 경우가 많기 때문

- 순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스 그룹에 속하는지 분류하기 위해, 뉴스 기사 내용 전처리하기
 - remove() 메서드 : 뉴스 그룹 기사의 헤터, 푸터 등 제거 가능
 - fetch_20newsgroups()의 subset 파라미터 : 학습 데이터 세트와 테스트 데이터 세트 분리해 내려받을 수 있음

In [6]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출
train_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_train = train_news.data
y_train = train_news.target

#subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출
test_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_test = train_news.data
y_test = train_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 11314


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

- 카운트 기반의 피처 벡터화 변환 수행하기

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 체스트 데이터를 피터 벡처화 변환 수행.
X_test_cnt_vect= cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 countVectorizer shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 countVectorizer shape: (11314, 101631)


- 카운트 기반으로 피처 벡터화된 데이터에 로지스틱 회귀 적용해 뉴스 그룹에 대한 분류 예측하기

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred= lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorized Logistic Regression의 예측 정확도는 0.964


- TF-IDF 기반으로 피처 벡터화 수행하고 로지스틱 회귀 예측 진행하기

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용해 학습/예측/평가 수행

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred= lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.907


- 좀 더 다양한 파라미터 적용하여 다시 예측 성능 측정하기

In [10]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

KeyboardInterrupt: ignored

- GridSearchCV 이용해 로지스틱 회귀의 하이퍼 파라미너 최적화 수행하기

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidt_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

In [ ]:
# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidt_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

### 사이킷런 파이프라인(Pipeline) 사용 GridSearchCV와의 결합

```
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression(random_state=156))])
```

- 기존 TfidVectorizer의 학습 데이터와 테스트 데이터에 대한 fit(), transform() 수행을 통한 피처 벡터화와 LogisticRegressor()dml fit()과 predict() 수행을 통한 머신 러닝 모델의 학습과 예측이 Pipeline의 fit()과 predict()로 통일되 수행되는 Pipeline 이용한 코드 작성하기
 - 효과 : 좀 더 직관적이고 쉽게 작성 가능

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfid_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
# pipeline의 fit과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

- Pipeline에서 하이퍼 파라미터를 함께 최적화하기

In [ ]:
from sklearn.model_selection import GridSearchCV

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))')